In [2]:
import pandas as pd

In [7]:
pd.read_csv("D:/uni/AdvancedTopics/project2\PossibileDB/IMDB/problem_sets.csv")

,T_validation,T_prime_validation,T_generalization,T_prime_generalization,Setup
0,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_4.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_5.csv,mixed_mixed_addition_columns
1,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_6.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_7.csv,mixed_numeric_addition_columns
2,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_8.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_9.csv,mixed_numeric_addition_columns_with_aggregations
3,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_10.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_11.csv,mixed_categorial_addition_columns
4,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_12.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_13.csv,only_textual_origins_mixed_addition_columns
5,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_14.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_15.csv,only_textual_origins_also_nlp_cleaning_mixed_a...
6,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_16.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_17.csv,only_textual_origins_onehot_addition_columns
7,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_18.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_19.csv,only_textual_origins_labelencoding_addition_co...
8,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_20.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_21.csv,only_textual_origins_frequencyencoding_additio...
9,IMDB/IMDB_Ver_0.csv,IMDB/IMDB_Ver_22.csv,IMDB/IMDB_Ver_1.csv,IMDB/IMDB_Ver_23.csv,only_textual_origins_numeric_targets_addition_...


In [3]:
pd.read_csv(r"D:\uni\AdvancedTopics\project2\PossibileDB\IMDB\IMDB_Ver_0.csv")

,Series_Title,Released_Year,Genre,Runtime,IMDB_Rating,Overview
0,Star Wars,1977,"Action, Adventure, Fantasy",121 min,8.6,Luke Skywalker joins forces with a Jedi Knight...
1,Dawn of the Dead,1978,"Action, Adventure, Horror",127 min,7.9,Following an ever-growing epidemic of zombies ...
2,The Day of the Jackal,1973,"Crime, Drama, Thriller",143 min,7.8,"A professional assassin codenamed ""Jackal"" plo..."
3,Harvey,1950,"Comedy, Drama, Fantasy",104 min,7.9,Due to his insistence that he has an invisible...
4,After Hours,1985,"Comedy, Crime, Drama",97 min,7.7,An ordinary word processor has the worst night...
...,...,...,...,...,...,...
795,Aliens,1986,"Action, Adventure, Sci-Fi",137 min,8.3,Fifty-seven years after surviving an apocalypt...
796,Kaze no tani no Naushika,1984,"Animation, Adventure, Fantasy",117 min,8.1,Warrior and pacifist Princess Nausicaä despera...
797,Zulu,1964,"Drama, History, War",138 min,7.7,Outnumbered British soldiers do battle with Zu...
798,La dolce vita,1960,"Comedy, Drama",174 min,8.0,A series of stories following a week in the li...


In [3]:
import pandas as pd
import numpy as np
import random
from faker import Faker

# Load the base IMDB table
base_df = pd.read_csv(r"D:\uni\AdvancedTopics\project2\PossibileDB\IMDB\IMDB_Ver_0.csv")

# Initialize Faker for realistic fake data
fake = Faker()

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

## Table 1: Movie Financials (could join on Series_Title)
num_movies = len(base_df)
movie_financials = pd.DataFrame({
    'Series_Title': base_df['Series_Title'],
    'Production_Budget': [random.randint(1, 200) * 1000000 for _ in range(num_movies)],
    'Box_Office_Gross': lambda x: [int(b * random.uniform(0.5, 10.0) for b in x['Production_Budget'])],
    'Profitability_Ratio': lambda x: x['Box_Office_Gross'] / x['Production_Budget'],
    'Primary_Production_Company': [fake.company() for _ in range(num_movies)],
    'Release_Season': [random.choice(['Spring', 'Summer', 'Fall', 'Winter']) for _ in range(num_movies)]
})

## Table 2: Director Information (could join on Series_Title)
directors = pd.DataFrame({
    'Series_Title': base_df['Series_Title'],
    'Director_Name': [fake.name() for _ in range(num_movies)],
    'Director_Gender': [random.choice(['Male', 'Female', 'Non-binary']) for _ in range(num_movies)],
    'Director_Nationality': [fake.country() for _ in range(num_movies)],
    'Director_Birth_Year': [random.randint(1940, 1990) for _ in range(num_movies)],
    'Director_Awards': [random.randint(0, 20) for _ in range(num_movies)]
})

## Table 3: Actor Information (many-to-many relationship, needs bridge table)
actors = pd.DataFrame({
    'Actor_ID': range(1, 201),
    'Actor_Name': [fake.name() for _ in range(200)],
    'Actor_Gender': [random.choice(['Male', 'Female']) for _ in range(200)],
    'Actor_Nationality': [fake.country() for _ in range(200)],
    'Actor_Debut_Year': [random.randint(1970, 2020) for _ in range(200)],
    'Actor_Awards': [random.randint(0, 15) for _ in range(200)]
})

# Bridge table for movie-actor relationships
movie_actor_bridge = pd.DataFrame({
    'Series_Title': random.choices(base_df['Series_Title'], k=500),
    'Actor_ID': random.choices(actors['Actor_ID'], k=500),
    'Role_Type': random.choices(['Lead', 'Supporting', 'Cameo'], weights=[0.5, 0.45, 0.05], k=500),
    'Salary': [random.randint(50000, 5000000) for _ in range(500)]
}).drop_duplicates()

## Table 4: Awards Information (could join on Series_Title)
awards = pd.DataFrame({
    'Series_Title': random.choices(base_df['Series_Title'], k=150),
    'Award_Name': random.choices(['Oscar', 'Golden Globe', 'BAFTA', 'Cannes', 'Sundance'], k=150),
    'Award_Category': random.choices(['Best Picture', 'Best Director', 'Best Actor', 'Best Actress', 
                                     'Best Screenplay', 'Best Cinematography'], k=150),
    'Award_Year': [random.randint(2000, 2023) for _ in range(150)],
    'Won': random.choices([True, False], weights=[0.3, 0.7], k=150)
}).drop_duplicates()

## Table 5: Streaming Availability (could join on Series_Title)
streaming = pd.DataFrame({
    'Series_Title': base_df['Series_Title'],
    'Available_On': [random.choice(['Netflix', 'Amazon Prime', 'HBO Max', 'Disney+', 'Hulu', 'None']) 
                     for _ in range(num_movies)],
    'Subscription_Required': [random.choice([True, False]) for _ in range(num_movies)],
    'Release_Date': [fake.date_between(start_date='-10y', end_date='today') for _ in range(num_movies)],
    'Price': [round(random.uniform(0, 19.99), 2) if random.random() > 0.7 else 0 for _ in range(num_movies)]
})

## Table 6: User Ratings (could join on Series_Title)
user_ratings = pd.DataFrame({
    'Series_Title': random.choices(base_df['Series_Title'], k=1000),
    'User_ID': [fake.uuid4() for _ in range(1000)],
    'User_Rating': [round(random.uniform(1, 10), 1) for _ in range(1000)],
    'Review_Date': [fake.date_between(start_date='-5y', end_date='today') for _ in range(1000)],
    'Review_Text': [fake.text(max_nb_chars=200) for _ in range(1000)]
})

## Table 7: Country-Specific Data (could join on Series_Title)
country_data = pd.DataFrame({
    'Series_Title': random.choices(base_df['Series_Title'], k=300),
    'Country': [fake.country() for _ in range(300)],
    'Release_Date': [fake.date_between(start_date='-30y', end_date='today') for _ in range(300)],
    'Localized_Title': [fake.catch_phrase() if random.random() > 0.7 else None for _ in range(300)],
    'Censorship_Rating': random.choices(['G', 'PG', 'PG-13', 'R', 'NC-17', 'Unrated'], k=300)
}).drop_duplicates()

# Save all tables to CSV files
tables = {
    'movie_financials': movie_financials,
    'directors': directors,
    'actors': actors,
    'movie_actor_bridge': movie_actor_bridge,
    'awards': awards,
    'streaming': streaming,
    'user_ratings': user_ratings,
    'country_data': country_data
}

for name, table in tables.items():
    table.to_csv(f'{name}.csv', index=False)

print("Generated synthetic tables:")
print("\n".join(tables.keys()))

Generated synthetic tables:
movie_financials
directors
actors
movie_actor_bridge
awards
streaming
user_ratings
country_data


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

# Set random seed for reproducibility
np.random.seed(42)

# Initialize ground truth dataframe
ground_truth = pd.DataFrame(columns=[
    'version_name',
    'version_number',
    'base_table',
    'joined_tables',
    'join_type',
    'missing_data_introduced',
    'missing_data_percentage',
    'description',
    'rows_in_output',
    'columns_in_output',
    'created_at'
])

# Function to record version metadata
def record_version(name, number, base, joins, join_type, missing, missing_pct, desc, df):
    new_record = pd.DataFrame([{
        'version_name': name,
        'version_number': number,
        'base_table': base,
        'joined_tables': joins,
        'join_type': join_type,
        'missing_data_introduced': missing,
        'missing_data_percentage': missing_pct,
        'description': desc,
        'rows_in_output': len(df),
        'columns_in_output': len(df.columns),
        'created_at': datetime.now()
    }])
    return new_record
# Function to introduce missing values
def add_missing_values(df, pct=0.1, exclude_cols=['Series_Title', 'Released_Year']):
    df_missing = df.copy()
    for col in df_missing.columns:
        if col not in exclude_cols:
            mask = np.random.random(len(df_missing)) < pct
            df_missing.loc[mask, col] = np.nan
    return df_missing

# Load all tables
base_df = pd.read_csv('sinteticDB/IMDB/IMDB_Base.csv')
movie_financials = pd.read_csv('sinteticDB/IMDB/movie_financials.csv')
directors = pd.read_csv('sinteticDB/IMDB/directors.csv')
streaming = pd.read_csv('sinteticDB/IMDB/streaming.csv')
country_data = pd.read_csv('sinteticDB/IMDB/country_data.csv')
awards = pd.read_csv('sinteticDB/IMDB/awards.csv')
user_ratings = pd.read_csv('sinteticDB/IMDB/user_ratings.csv')
movie_actor_bridge = pd.read_csv('sinteticDB/IMDB/movie_actor_bridge.csv')
actors = pd.read_csv('sinteticDB/IMDB/actors.csv')

# Define missing value percentages to simulate
missing_percentages = [0.05, 0.15]  # 5% and 15% missing data

# Version counter
version_counter = 1

# Original versions without missing data
versions = [
    # Version 1: Base + Financials
    ('imdb_with_financials', 
     lambda: pd.merge(base_df, movie_financials, on='Series_Title', how='left'),
     "Base table with financial information added"),
    
    # Version 2: Base + Directors
    ('imdb_with_directors', 
     lambda: pd.merge(base_df, directors, on='Series_Title', how='left'),
     "Base table with director information added"),
    
    # Version 3: Base + Streaming
    ('imdb_with_streaming', 
     lambda: pd.merge(base_df, streaming, on='Series_Title', how='left'),
     "Base table with streaming availability added"),
    
    # Version 4: Base + Country Data
    ('imdb_with_country_data', 
     lambda: pd.merge(base_df, country_data, on='Series_Title', how='left'),
     "Base table with country-specific release data added"),
    
    # Version 5: Base + Awards
    ('imdb_with_awards', 
     lambda: pd.merge(base_df, awards, on='Series_Title', how='left'),
     "Base table with awards information (may have duplicate movie rows)"),
    
    # Version 6: Base + Aggregated User Ratings
    ('imdb_with_ratings', 
     lambda: pd.merge(
         base_df, 
         user_ratings.groupby('Series_Title')['User_Rating'].agg(['mean', 'count']).reset_index().rename(columns={'mean': 'Avg_User_Rating', 'count': 'Rating_Count'}),
         on='Series_Title', how='left'),
     "Base table with average user ratings added"),
    
    # Version 7: Base + Actor Count
    ('imdb_with_actor_count', 
     lambda: pd.merge(
         base_df,
         movie_actor_bridge.groupby('Series_Title')['Actor_ID'].count().reset_index().rename(columns={'Actor_ID': 'Actor_Count'}),
         on='Series_Title', how='left'),
     "Base table with count of actors per movie"),
    
    # Version 8: Base + Lead Actors
    ('imdb_with_lead_actor', 
     lambda: pd.merge(
         base_df,
         pd.merge(
             movie_actor_bridge[movie_actor_bridge['Role_Type'] == 'Lead'].drop_duplicates('Series_Title', keep='first'),
             actors,
             on='Actor_ID'
         )[['Series_Title', 'Actor_Name']],
         on='Series_Title', how='left'),
     "Base table with name of lead actor added"),
    
    # Version 9: Base + Financials + Directors
    ('imdb_with_financials_and_directors', 
     lambda: pd.merge(
         pd.merge(base_df, movie_financials, on='Series_Title', how='left'),
         directors,
         on='Series_Title', how='left'),
     "Base table with both financial and director information"),
    
    # Version 10: Comprehensive (will be handled separately)
]

# Create original versions and missing data variants
for version_name, join_func, desc in versions:
    # Create original version
    df = join_func()
    df.to_csv(f'{version_name}.csv', index=False)
    ground_truth = pd.concat([
        ground_truth,
        record_version(
            version_name, version_counter,
            'IMDB_Base.csv', version_name.split('_with_')[1],
            'left', False, 0,
            desc, df
        )
    ], ignore_index=True)
    version_counter += 1
    
    # Create versions with missing values
    for pct in missing_percentages:
        missing_version_name = f"{version_name}_missing_{int(pct*100)}pct"
        df_missing = add_missing_values(df, pct)
        df_missing.to_csv(f'{missing_version_name}.csv', index=False)
        ground_truth = pd.concat([
            ground_truth,
            record_version(
                missing_version_name, version_counter,
                'IMDB_Base.csv', version_name.split('_with_')[1],
                'left', True, pct,
                f"{desc} with {int(pct*100)}% missing values",
                df_missing
            )
        ], ignore_index=True)
        version_counter += 1

# Special case for comprehensive version
comp_df = pd.merge(base_df, movie_financials, on='Series_Title', how='left')
comp_df = pd.merge(comp_df, streaming, on='Series_Title', how='left')
comp_df = pd.merge(comp_df, 
                  user_ratings.groupby('Series_Title')['User_Rating'].mean().reset_index(), 
                  on='Series_Title', how='left')
comp_df.to_csv('imdb_comprehensive.csv', index=False)
ground_truth = pd.concat([
    ground_truth,
    record_version(
        'imdb_comprehensive', version_counter,
        'IMDB_Base.csv', 'movie_financials + streaming + user_ratings',
        'left', False, 0,
        "Base table with financials, streaming, and ratings combined",
        comp_df
    )
], ignore_index=True)
version_counter += 1

# Create missing versions for comprehensive
for pct in missing_percentages:
    missing_version_name = f"imdb_comprehensive_missing_{int(pct*100)}pct"
    comp_missing = add_missing_values(comp_df, pct)
    comp_missing.to_csv(f'{missing_version_name}.csv', index=False)
    ground_truth = pd.concat([
        ground_truth,
        record_version(
            missing_version_name, version_counter,
            'IMDB_Base.csv', 'movie_financials + streaming + user_ratings',
            'left', True, pct,
            f"Comprehensive version with {int(pct*100)}% missing values",
            comp_missing
        )
    ], ignore_index=True)
    version_counter += 1

# Save ground truth
ground_truth.to_csv('ground_truth_versions.csv', index=False)

print(f"Created {len(ground_truth)} total versions")
print(f"Saved ground truth to: ground_truth_versions.csv")

C:\Users\Elia\AppData\Local\Temp\ipykernel_22704\1552721740.py:136: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ground_truth = pd.concat([
C:\Users\Elia\AppData\Local\Temp\ipykernel_22704\1552721740.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_missing.loc[mask, col] = np.nan
C:\Users\Elia\AppData\Local\Temp\ipykernel_22704\1552721740.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first

Created 30 total versions
Saved ground truth to: ground_truth_versions.csv


C:\Users\Elia\AppData\Local\Temp\ipykernel_22704\1552721740.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_missing.loc[mask, col] = np.nan
C:\Users\Elia\AppData\Local\Temp\ipykernel_22704\1552721740.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_missing.loc[mask, col] = np.nan


In [11]:
ground_truth

,version_name,version_number,base_table,joined_tables,join_type,missing_data_introduced,missing_data_percentage,description,rows_in_output,columns_in_output,created_at
0,imdb_with_financials,1,IMDB_Base.csv,financials,left,False,0,Base table with financial information added,800,11,2025-04-03 19:52:49.579344
1,imdb_with_financials_missing_5pct,2,IMDB_Base.csv,financials,left,True,0.05,Base table with financial information added wi...,800,11,2025-04-03 19:52:49.592819
2,imdb_with_financials_missing_15pct,3,IMDB_Base.csv,financials,left,True,0.15,Base table with financial information added wi...,800,11,2025-04-03 19:52:49.603050
3,imdb_with_directors,4,IMDB_Base.csv,directors,left,False,0,Base table with director information added,800,11,2025-04-03 19:52:49.630909
4,imdb_with_directors_missing_5pct,5,IMDB_Base.csv,directors,left,True,0.05,Base table with director information added wit...,800,11,2025-04-03 19:52:49.640908
5,imdb_with_directors_missing_15pct,6,IMDB_Base.csv,directors,left,True,0.15,Base table with director information added wit...,800,11,2025-04-03 19:52:49.650158
6,imdb_with_streaming,7,IMDB_Base.csv,streaming,left,False,0,Base table with streaming availability added,800,10,2025-04-03 19:52:49.659081
7,imdb_with_streaming_missing_5pct,8,IMDB_Base.csv,streaming,left,True,0.05,Base table with streaming availability added w...,800,10,2025-04-03 19:52:49.668311
8,imdb_with_streaming_missing_15pct,9,IMDB_Base.csv,streaming,left,True,0.15,Base table with streaming availability added w...,800,10,2025-04-03 19:52:49.677402
9,imdb_with_country_data,10,IMDB_Base.csv,country_data,left,False,0,Base table with country-specific release data ...,847,10,2025-04-03 19:52:49.685596


In [15]:
pd.read_csv(r"D:\uni\AdvancedTopics\project2\sinteticDB\IMDB\Versions\imdb_with_actor_count.csv")

,Series_Title,Released_Year,Genre,Runtime,IMDB_Rating,Overview,Actor_Count
0,Star Wars,1977,"Action, Adventure, Fantasy",121 min,8.6,Luke Skywalker joins forces with a Jedi Knight...,NaN
1,Dawn of the Dead,1978,"Action, Adventure, Horror",127 min,7.9,Following an ever-growing epidemic of zombies ...,1.0
2,The Day of the Jackal,1973,"Crime, Drama, Thriller",143 min,7.8,"A professional assassin codenamed ""Jackal"" plo...",NaN
3,Harvey,1950,"Comedy, Drama, Fantasy",104 min,7.9,Due to his insistence that he has an invisible...,1.0
4,After Hours,1985,"Comedy, Crime, Drama",97 min,7.7,An ordinary word processor has the worst night...,2.0
...,...,...,...,...,...,...,...
795,Aliens,1986,"Action, Adventure, Sci-Fi",137 min,8.3,Fifty-seven years after surviving an apocalypt...,NaN
796,Kaze no tani no Naushika,1984,"Animation, Adventure, Fantasy",117 min,8.1,Warrior and pacifist Princess Nausicaä despera...,NaN
797,Zulu,1964,"Drama, History, War",138 min,7.7,Outnumbered British soldiers do battle with Zu...,NaN
798,La dolce vita,1960,"Comedy, Drama",174 min,8.0,A series of stories following a week in the li...,NaN


In [3]:
pd.read_csv(r"D:\uni\AdvancedTopics\project2\sinteticDB\IMDB\Versions\imdb_with_awards.csv")

,Series_Title,Released_Year,Genre,Runtime,IMDB_Rating,Overview,Award_Name,Award_Category,Award_Year,Won
0,Star Wars,1977,"Action, Adventure, Fantasy",121 min,8.6,Luke Skywalker joins forces with a Jedi Knight...,Oscar,Best Director,2006.0,False
1,Dawn of the Dead,1978,"Action, Adventure, Horror",127 min,7.9,Following an ever-growing epidemic of zombies ...,NaN,NaN,NaN,NaN
2,The Day of the Jackal,1973,"Crime, Drama, Thriller",143 min,7.8,"A professional assassin codenamed ""Jackal"" plo...",Oscar,Best Actor,2000.0,True
3,Harvey,1950,"Comedy, Drama, Fantasy",104 min,7.9,Due to his insistence that he has an invisible...,NaN,NaN,NaN,NaN
4,After Hours,1985,"Comedy, Crime, Drama",97 min,7.7,An ordinary word processor has the worst night...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
809,Kaze no tani no Naushika,1984,"Animation, Adventure, Fantasy",117 min,8.1,Warrior and pacifist Princess Nausicaä despera...,Golden Globe,Best Cinematography,2014.0,False
810,Zulu,1964,"Drama, History, War",138 min,7.7,Outnumbered British soldiers do battle with Zu...,BAFTA,Best Cinematography,2012.0,False
811,Zulu,1964,"Drama, History, War",138 min,7.7,Outnumbered British soldiers do battle with Zu...,Golden Globe,Best Actress,2005.0,True
812,La dolce vita,1960,"Comedy, Drama",174 min,8.0,A series of stories following a week in the li...,NaN,NaN,NaN,NaN


In [7]:
prova=pd.read_csv(r"D:\uni\AdvancedTopics\project2\sinteticDB\IMDB\externalTables\awards.csv")

In [13]:
result = prova[prova['Series_Title'] == "Star Wars"]
print(result)

   Series_Title Award_Name Award_Category  Award_Year    Won
93    Star Wars      Oscar  Best Director        2006  False
